<a href="https://colab.research.google.com/github/lustraka/puzzles/blob/main/AoC2021/AoC_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advent of Code Puzzles
[Advent of Code 2021](https://adventofcode.com/2021) | [reddit/adventofcode](https://www.reddit.com/r/adventofcode/)

In [1]:
import requests
import pandas as pd
import numpy as np
path = 'https://raw.githubusercontent.com/lustraka/puzzles/main/AoC2021/data/'

## [Day 9](https://adventofcode.com/2021/day/9): Smoke Basin
### Part I
- **Unknown**: A sum of the risk levels of all low points on a heightmap.
- **Data**: A heightmap $n \times ?$
- **Condition**:
  - Diagonal locations do not count as adjacent.
  - The risk level of a low point is 1 plus its height.
- **Plan**:
  - Create a matrix
  - Bound the matrix with `9` to simplify search over location using indices
  - Initialize the sum of the risk levels `rl_sum` = 0
  - Iterate over rows, check adjucent locations, adjust `rl_sum` in locally lowest points.

In [2]:
example = """2199943210
3987894921
9856789892
8767896789
9899965678"""

In [30]:
def parse(data):
  # Read the baseline map
  hmap_bl = np.array([list(row) for row in data.split()]).astype(int)
  # Add vertical boundaries
  b = 9*np.ones(hmap_bl.shape[1]).reshape(1,-1).astype(int)
  hmap = np.vstack((b,hmap_bl,b))
  # Add horizontal boundaries
  b = 9*np.ones(hmap.shape[0]).reshape(-1,1).astype(int)
  hmap_tg = np.hstack((b,hmap,b))

  return hmap_tg
hmap = parse(example)
hmap

array([[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9],
       [9, 2, 1, 9, 9, 9, 4, 3, 2, 1, 0, 9],
       [9, 3, 9, 8, 7, 8, 9, 4, 9, 2, 1, 9],
       [9, 9, 8, 5, 6, 7, 8, 9, 8, 9, 2, 9],
       [9, 8, 7, 6, 7, 8, 9, 6, 7, 8, 9, 9],
       [9, 9, 8, 9, 9, 9, 6, 5, 6, 7, 8, 9],
       [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]])

In [46]:
def solve_part1(input):
  """Sum the risk levels of all low points on a heightmap."""
  
  # Read the hightmap (with boundary)
  hmap = parse(input)
  # Identify the shape of bounded map
  r,c = hmap.shape
  print(f"hmap.shape = {hmap.shape}")
  # Initialize sum of risk levels
  rl_sum = 0
  # Iterate over map to find local minima
  for i in range(1,r-1):
    for j in range(1,c-1):
      if hmap[i,j] == min(
          hmap[i,j],
          hmap[i-1,j],
          hmap[i+1,j],
          hmap[i,j-1],
          hmap[i,j+1]
      ) and hmap[i,j] != 9:
        rl_sum += hmap[i,j]+1
        # print(f"hmap[{i},{j}] = {hmap[i,j]}, rl_sum = {rl_sum}.")
  return rl_sum

solve_part1(example)

hmap.shape = (7, 12)


15

In [36]:
r = requests.get(path+'AoC2021_09.txt')
hmap = parse(r.text[:-1])
hmap[-2:]

array([[9, 2, 1, 2, 3, 4, 5, 6, 9, 9, 9, 8, 7, 6, 5, 6, 7, 8, 9, 9, 8, 7,
        5, 1, 0, 1, 2, 5, 6, 7, 8, 9, 1, 0, 9, 9, 9, 9, 9, 8, 4, 3, 2, 1,
        9, 8, 9, 9, 8, 9, 8, 9, 9, 5, 4, 3, 2, 4, 9, 8, 7, 8, 9, 4, 5, 6,
        7, 8, 9, 9, 6, 5, 4, 2, 1, 0, 1, 9, 8, 9, 8, 7, 7, 6, 6, 7, 8, 9,
        9, 8, 7, 8, 9, 8, 9, 1, 2, 3, 6, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
        9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
        9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
        9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
        9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]])

In [44]:
hmap[0:3,35:38]

array([[9, 9, 9],
       [9, 9, 9],
       [8, 9, 8]])

In [47]:
solve_part1(r.text[:-1])

hmap.shape = (102, 102)


572

## Part II
- **Unknown**: The product of the sizes of the three largest basins.
- **Data**: The same heightmap as in the part I.
- **Condition**: Locations of height 9 do not count as being in any basin (these are boundaries of basins), and all other locations will always be part of exactly one basin.
- **Plan**:
  - Construct a Basin object with methods
    - \_\_init__() (initializes the lowest point)
    - add_location(), 
    - get_size()
    - explore_hmap() (uses recursion to map the whole basin)
    - \_\_lt__() (to enable sorting according to size)
    - \_\_repr__()
  - Map basins in the heightmap
    - Begin with the hight = 0
    - Explore all basins with this bottom
    - Increment hight and repeat over all so far not explored locations up to hight 7
  - Determine size of the three largest basins and return their product.

In [100]:
class Basin():
  def __init__(self, lowest):
    self.locs = [lowest]
    self.bottom = lowest

  def add_location(self, loc):
    self.locs.append(loc)
  
  def check_location(self, loc):
    """Return True if loc in this basin."""
    return loc in self.locs
  
  def get_size(self):
    return len(self.locs)
  
  def explore_hmap(self, hmap, loc):
    """For all directions if hight is not 9, add location and explore it."""
    
    # Initialize coordinates
    r,c = loc
    for i,j in [[r-1,c], [r+1,c],[r,c-1],[r,c+1]]:
      # If adjucent location is not in the basin or in the boundary
      if not self.check_location([i,j]) and hmap[i,j] != 9:
        # Add location to the basin
        self.add_location([i,j])
        # Explore location recurively
        self.explore_hmap(hmap, [i,j])

  def __lt__(self, other):
    return self.get_size() < other.get_size()

  def __repr__(self):
    return f"Basin(bottom={self.bottom}, size={self.get_size()})"

In [113]:
def solve_part2(input):
  """Explore basins"""

  # Read the hightmap with boundary
  hmap = parse(input)
  # Identify the shape of bounded map
  r,c = hmap.shape
  print(f"hmap.shape = {hmap.shape}")
  # Initialize list of basins
  basins = []
  # Iterate over hights from 0 to 7
  for h in range(8):
    # Iterate over map to find local minima
    for i in range(1,r-1):
      for j in range(1,c-1):
        in_some_basin = False
        for basin in basins:
          if basin.check_location([i,j]):
            in_some_basin = True
        if hmap[i,j] == h and not in_some_basin:
          b = Basin([i,j])
          basins.append(b)
          # When local bottom found, explore it
          b.explore_hmap(hmap,[i,j])
  
  print(f"The number of explored basins is {len(basins)}.")
  print(f"The three largest basins: {sorted(basins, reverse=True)[:3]}")

  # Take sizes of the three largest basins
  sb = [b.get_size() for b in sorted(basins, reverse=True)[:3]]
  
  # Multiply the sizes
  return sb[0]*sb[1]*sb[2]

solve_part2(example)

hmap.shape = (7, 12)
The number of explored basins is 4.
The three largest basins: [Basin(bottom=[3, 3], size=14), Basin(bottom=[1, 10], size=9), Basin(bottom=[5, 7], size=9)]


1134

In [114]:
solve_part2(r.text[:-1])

hmap.shape = (102, 102)
The number of explored basins is 242.
The three largest basins: [Basin(bottom=[79, 52], size=99), Basin(bottom=[29, 15], size=93), Basin(bottom=[52, 48], size=92)]


847044

## [Day 8](https://adventofcode.com/2021/day/8): Seven Segment Search
### Part I
- **Unknown**: Count of the *easy* digits with unique pattern (i.e. 1, 4, 7, 8) in the output.
- **Data**: a number of entries with 10 unique signal patterns and 4 digit output value separated by a `|` delimiter.

To solve part I, I need to sum strings of length either 2 (digit 1), 3 (digit 7), 4 (digit 4), or 8 (digit 8) in the output.

In [ ]:
example = """be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce"""

In [ ]:
# A dictionary of easy digits and their counts
digie = {1:0, 4:0, 7:0, 8:0}

In [ ]:
# To add 1 to a digie counter
digie[1] +=1
digie

{1: 1, 4: 0, 7: 0, 8: 0}

In [ ]:
# Count values in dictionary using dictionary
def count_easy_digits(digie, digit):
  """Add 1 to the relevant digie key accoding to the length
  of the digit value."""

  x = len(digit)
  if x == 2:
    digie[1] += 1
  elif x == 3:
    digie[7] += 1
  elif x == 4:
    digie[4] += 1
  elif x == 7:
    digie[8] += 1
 
  return 

In [ ]:
# Test the function
count_easy_digits(digie, 'cbce')
print(digie)
sum(digie.values())

{1: 1, 4: 1, 7: 0, 8: 0}


2

In [ ]:
# Define the parse function
def parse(input):
  """Transform string to list of entries.
  Each entry consists of 10 signal patterns
  and four digit output value in two lists."""

  entries = [e.split('|') for e in input.split('\n')]
  for entry in entries:
    entry[0] = entry[0].split()
    entry[1] = entry[1].split()

  return entries

data = parse(example)
data[-1]

[['gcafb',
  'gcf',
  'dcaebfg',
  'ecagb',
  'gf',
  'abcdeg',
  'gaef',
  'cafbge',
  'fdbac',
  'fegbdc'],
 ['fgae', 'cfgab', 'fg', 'bagce']]

In [ ]:
r = requests.get(path+'AoC2021_08.txt')
parse(r.text[:-1])[-1]

[['fcedg',
  'cfdaegb',
  'dbfg',
  'egcfbd',
  'bgc',
  'cgbefa',
  'cebgd',
  'ecbad',
  'bg',
  'dceafg'],
 ['cgb', 'fcbgae', 'ecbda', 'gebcd']]

Is this a dead end?
```python
# Count values in dictionary using dictionary
def count_easy_digits(digie, s):
  """Add 1 to the relevant digie key."""

  return {
      2 : lambda: digie[1] + 1,
      3 : lambda: digie[7] + 1,
      4 : lambda: digie[4] + 1,
      8 : lambda: digie[8] + 1,
  }.get(len(s), lambda: None)()
```

In [ ]:
def solve_part1(input):
  """Count easy digits in the output value."""
  data = parse(input)
  
  # A dictionary of easy digits and their counts
  digie = {1:0, 4:0, 7:0, 8:0}

  for entry in data:
    for digit in entry[1]:
      count_easy_digits(digie, digit)
  
  return sum(digie.values())

In [ ]:
solve_part1(example)

26

In [ ]:
solve_part1(r.text[:-1])

321

### Part II
- **Unknown**: Sum of decoded four digit seven-segment display's output values.
- **Data**: A number of entries with 10 unique signal patterns and 4 digit output value separated by a `|` delimiter.
- **Condition**: The mapping between signal wires and segments for each entry derived from signal pattern.

To solve the puzzle:
- for `entry` in `entries`:
  - decode mapping between signal wires a to g and segments
  - decode digits on the display
  - transform digits into an integer value
- sum values of all entries

> **Hint**: I used sets. I converted each input to a set of characters and using set operations (difference, union, etc) you can determine any value using the set for four or one, which are trivial to calculate using length. Super easy and you can solve the whole thing once you've found those two sets. [PM_ME_YOUR_MECH](https://www.reddit.com/r/adventofcode/comments/rbj87a/2021_day_8_solutions/hnp3utf/?context=3)

In [ ]:
data = parse('acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | cdfeb fcadb cdfeb cdbaf')
# A unique signal patterns
usps = sorted([set(p) for p in data[0][0]], key=len)
for i,s in enumerate(usps):
  print(i,sorted(s))

0 ['a', 'b']
1 ['a', 'b', 'd']
2 ['a', 'b', 'e', 'f']
3 ['b', 'c', 'd', 'e', 'f']
4 ['a', 'c', 'd', 'f', 'g']
5 ['a', 'b', 'c', 'd', 'f']
6 ['a', 'b', 'c', 'd', 'e', 'f']
7 ['b', 'c', 'd', 'e', 'f', 'g']
8 ['a', 'b', 'c', 'd', 'e', 'g']
9 ['a', 'b', 'c', 'd', 'e', 'f', 'g']


- The list `entry_0` holds the first part of the entry.
- The list `usps` holds patterns sorted by length.
- The list `dmap` holds patterns according their value.
- The function `map_digits()` takes unique string patterns and sorts them to `map` by their decoded values.

The function `decode_digit()` returns the index of a set which is equivalent to `code` in input.

In [ ]:
def map_digits(entry_0):
  """Sort patterns according their value."""
  usps = sorted([set(p) for p in entry_0], key=len)
  dmap = ['', usps[0], '', '', usps[2], '', '', usps[1], usps[9], '']
  penta = [3,4,5]
  # The digit 5 
  for i in penta:
    if usps[2].difference(usps[0]).issubset(usps[i]):
      dmap[5] = usps[i]
      penta.pop(penta.index(i))
  # The digit 3
  for i in penta:
    if usps[0].issubset(usps[i]):
      dmap[3] = usps[i]
      penta.pop(penta.index(i))
  # The digit 2 is last in penta index
  dmap[2] = usps[penta[0]]

  hexa = [6,7,8]
  # The digit 0 
  for i in hexa:
    if not usps[2].difference(usps[0]).issubset(usps[i]):
      dmap[0] = usps[i]
      hexa.pop(hexa.index(i))
  # The digit 9
  for i in hexa:
    if usps[0].issubset(usps[i]):
      dmap[9] = usps[i]
      hexa.pop(hexa.index(i))
  # The digit 6 is last in hexa index
  dmap[6] = usps[hexa[0]]

  return dmap

# Use dmap to decode the output
def decode_digit(dmap, code):
  """Return an index of `code` in `dmap`."""
  for i in range(10):
    if dmap[i] == set(code):
      return i

def solve_part2(input):
  """Map digits in patterns, transform outputs to values, return sum of values."""

  entries = parse(input)
  # Initialize the output variable
  sum = 0
  # Iterate over entries to get the value
  for entry in entries:
    # Decode patterns
    dmap = map_digits(entry[0])

    # Decode output
    value = int(''.join(map(str,[decode_digit(dmap, d) for d in entry[1]])))
    sum += value

  return sum

In [ ]:
solve_part2(example)

61229

In [ ]:
solve_part2(r.text[:-1])

1028926